In [ ]:
import numpy as np
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F


from tqdm import tqdm
import PIL.Image as Image

if not os.path.isdir('./experiments'):
    os.makedirs('./experiments')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Data Loading & Model creation

In [ ]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256), Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

train_transforms = transforms.Compose([
    transforms.Resize((256, 256), Image.BILINEAR),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize    
])

valid_transforms = transforms.Compose([
    transforms.Resize((256, 256), Image.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

batch_size = 64

train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('../input/mva-recvis-2021/bird_dataset/train_images', transform=train_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('../input/mva-recvis-2021/bird_dataset/val_images',transform=valid_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('../input/mva-recvis-2021/bird_dataset/test_images',transform=valid_transforms),
    batch_size=1, shuffle=False, num_workers=1)

train_size, val_size = len(train_loader.dataset), len(val_loader.dataset)
print(train_size, val_size)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()


for f in os.listdir('experiments/'):
    os.remove(os.path.join('experiments', f))



model_name = 'resnet152'
# model_name = 'efficientnet_b7'
# model_name = 'resnext'

if model_name == 'resnet152':
    model = torchvision.models.resnet152(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
#     for param in model.layer4.parameters():
#         param.requires_grad = True
    # for param in model.fc.parameters():
    #     param.requires_grad = True   
    model.fc = nn.Linear(model.fc.in_features, 20)


if model_name == 'efficientnet_b7':
    model = torchvision.models.efficientnet_b7(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    model.classifier()

if model_name == 'resnext':
    model = torchvision.models.resnext101_32x8d(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    for param in model.layer4.parameters():
        param.requires_grad = True
    model.fc = nn.Linear(model.fc.in_features, 20)



In [ ]:
# for param in model.parameters():
#     if param.requires_grad:
#         print(param.size())

# model

In [ ]:
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5, weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, min_lr=1e-8, patience=10)
criterion = torch.nn.CrossEntropyLoss()

# Train

In [ ]:
def train(model, epoch):
    model.train()
    training_loss = 0
    correct = 0
    
    for batch_idx, (data, labels) in enumerate(train_loader):
               
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        
        #forward
        preds = model(data)
        loss = criterion(preds, labels)
        
        training_loss += loss.data.cpu().item()*len(data)
        loss.backward()
        optimizer.step()
        
        probs = F.softmax(preds, dim=1)
        preds_classes = probs.max(1)[1]
        correct += (preds_classes == labels).sum().data.cpu().detach().item()
        
        if batch_idx % 25 == 0:
            print('[{:4d}/{:4d} ({:2.0f}%)]\tLoss: {:.4f}'.format(
                batch_idx * batch_size, train_size,
                100. * batch_idx * batch_size / train_size, loss.data.cpu().detach().item()))
    
    return training_loss / train_size, correct / train_size


def validation(model):
    model.eval()
    validation_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            preds = model(data)
            
            # sum up batch loss
            validation_loss += criterion(preds, labels).data.cpu().detach().item()*len(data)
            probs = F.softmax(preds, dim=1)
            preds_classes = probs.max(1)[1]
            correct += (preds_classes == labels).sum().data.cpu().detach().item()
            
    return validation_loss / val_size, correct / val_size

In [ ]:
epochs = 25
training_losses = []
training_accs = []
validation_losses = []
validation_accs = []

best_acc = 0.

for epoch in range(1, epochs + 1):
    print("\n################################################# EPOCH", epoch)
    training_loss, training_acc = train(model, epoch)
    validation_loss, validation_acc = validation(model)
    
    training_losses.append(training_loss)
    training_accs.append(training_acc)
    validation_losses.append(validation_loss)
    validation_accs.append(validation_acc)
    
    scheduler.step(validation_loss)
    
    print('Training set:\t Average loss: {:.4f}\t Accuracy: {:.0f}/{:.0f} ({:.0f}%)'.format(
        training_loss, training_acc*train_size, train_size, training_acc*100))
    
    print('Validation set:\t Average loss: {:.4f}\t Accuracy: {:.0f}/{:.0f} ({:.0f}%)'.format(
        validation_loss, validation_acc*val_size, val_size, validation_acc*100))
    
    if validation_acc >= best_acc or epoch==epochs:
        print('\n**********Saving model with accuracy {:0.4f} at epoch {:2d}'.format(validation_acc, epoch))
        best_acc = validation_acc
        model_file = 'experiments' + '/model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12.8, 6.4))
its = np.arange(0, epochs)
plt.plot(its, training_losses[its[0]:], label='train')
plt.plot(its, validation_losses[its[0]:], label='valid')
plt.legend(loc=2)

plt.sca(plt.gca().twinx())
plt.plot(its, training_accs[its[0]:], '--', label='train acc')
plt.plot(its, validation_accs[its[0]:], '--', label='valid acc')

plt.title(model_name)
plt.legend(loc=7)

plt.show()

In [ ]:
np.argmin(validation_losses), np.argmax(validation_accs)

# Test

In [ ]:
preds = np.array([])

model.load_state_dict(torch.load('experiments/model_' + str(25) + '.pth'))

model.eval()
with torch.no_grad():
    for i, (data, labels) in tqdm(enumerate(test_loader, 0)):
        data, labels = data.to(device), labels.to(device)
        output1 = model(data)
        sm = nn.Softmax(dim=1)(output1)
        pred = sm.max(1, keepdim=True)[1]    
        preds = np.hstack((preds, torch.squeeze(pred).cpu().numpy()))

In [ ]:
f = open("submission.csv", "w")
f.write("Id,Category\n")
for (n,_),p in zip(test_loader.dataset.samples,preds):
    f.write("{},{}\n".format(n.split('/')[-1].split('.')[0], int(p)))
f.close()